In [ ]:
!pip install openai --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.0/383.0 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.6 MB/s eta 0:00:00


In [ ]:
import json
import csv
import pandas as pd
from openai import OpenAI

# api key 입력
client = OpenAI(api_key="자신의 키를 입력해주세요.")

In [ ]:
# JSON 파일 읽기
with open('crawl_kci.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

# 제목 추출
titles = [item['제목'] for item in data.values()]
# 초록 추출
abstracts = [item['초록'] for item in data.values()]

# DataFrame 생성 (ID 열 추가)
df = pd.DataFrame({'ID': range(len(titles)), '제목': titles})
df_with_abstracts = pd.DataFrame({'ID': range(len(titles)), '제목': titles, '초록': abstracts})

# 결과 출력
print(df)

# CSV 파일로 저장 (선택사항)
df.to_csv('kci_titles.csv', index=False, encoding='utf-8-sig')
df_with_abstracts.to_csv('kci_titles_with_abstracts.csv', index=False, encoding='utf-8-sig')

In [ ]:
# kci_titles.csv 를 df에 넣기
df = pd.read_csv('kci_titles.csv')

# df에서 10000개씩 여러개 추출하기
chunk_size = 10000
my_chunks = [df[i:i+chunk_size] for i in range(0, len(df), chunk_size)]
len(my_chunks)

14

# 1차 분류하기 (10개씩, 병렬처리)

In [ ]:
from pydantic import BaseModel

class Categorize(BaseModel):
    category: str

In [ ]:
def classify_kci_category(title):

    # 사회과학, 기독교신학, 문학, 언어학, 역사학, 철학, 통역번역학, 기타인문학, 관련없음 ('인문학이아님' 으로 수정하는 건?)
    # 기독교신학부터 기타인문학까지는 kci의 분류를 참고. 영어영문학, 국어국문학 => 문학, 언어학으로 크게 분류
    # 사회과학, 관련없음에 해당하는 것은 제외를 하려고 추가

    prompt = f"""다음 학술 논문 제목을 바탕으로 KCI (한국학술지인용색인) 카테고리를 분류해주세요.
    가능한 카테고리는 다음과 같습니다:
    사회과학, 기독교신학, 문학, 언어학, 역사학, 철학, 통역번역학
    인문학이지만 위 카테고리에 해당하지 않으면 '기타인문학'으로 분류하세요.
    위와 같은 인문학 카테고리와 아무 관련도 없다면 '관련없음'으로 분류하세요.
    '///'로 분류된 10개의 제목에 대해 각각 카테고리를 분류해주세요.

    <예시>
    제목: 속성의 의미를 나타내는 피동문의 특성 연구///부정성의 극단화로서의 노년: 노년의 철학적 규정에 대한 연구///고통의 시대와 저항담론으로서의 불교사상///중국 역사교과서의 개편과 자국사 및 세계사의 '현대' 서술///러시아 탈북민의 복음수용에 관한 생애사 연구///미시마 유키오 『한여름의 죽음』에서 보는 '분노'///밀의 공리주의와 덕 윤리학///1930년대 초반 프롤레타리아 소설의 계급의식과 여성 동성사회성///'계급' 개념의 근대 지식적 역학－사회주의 연구노트 1///줄기세포기반치료에 대한 미국의 규제 경향 고찰

    카테고리: 언어학///철학///철학///역사학///기독교신학///문학///철학///문학///사회과학///관련없음

    <실전>
    제목: {title[0]}///{title[1]}///{title[2]}///{title[3]}///{title[4]}///{title[5]}///{title[6]}///{title[7]}///{title[8]}///{title[9]}

    카테고리:"""

    # # old prompt는 분류가 안된 것들에 한해서  (13만개 중 1800개가 10개씩 분류할 때 분류가 안됨.)

    # 카테고리:"""

    # old_prompt = '''다음 학술 논문 제목을 바탕으로 KCI (한국학술지인용색인) 카테고리를 분류해주세요.
    # 가능한 카테고리는 다음과 같습니다:
    # 사회과학, 기독교신학, 문학, 언어학, 역사학, 철학, 통역번역학
    # 인문학이지만 위 카테고리에 해당하지 않으면 '기타인문학'으로 분류하세요.
    # 위와 같은 인문학 카테고리와 아무 관련도 없다면 '관련없음'으로 분류하세요.

    # 제목: {title}

    # 카테고리:'''

    try:
        response = client.beta.chat.completions.parse(
            model="gpt-4o-mini",
            temperature=0,
            messages=[
                {"role": "system", "content": "너는 카테고리를 정확하게 분류하는 AI야. 잘 고민하고 분류해봐."},
                {"role": "user", "content": prompt}
            ],
            response_format=Categorize,
        )

        if response.choices and response.choices[0].message.parsed:
            return response.choices[0].message.parsed.category
        else:
            return "분류 불가"  # 응답 구조가 올바르지 않은 경우
    except Exception as e:
        print(f"카테고리 분류 중 오류 발생: {e}")
        return "오류 발생"  # 오류 발생 시

In [ ]:
from tqdm import tqdm
import concurrent.futures
from functools import partial

def process_titles(titles):
    categories = classify_kci_category(titles).split('///')
    return categories

def process_chunk(chunk, titles_size, max_workers=20,):
    results = []
    title_groups = [chunk['제목'].iloc[i:i+titles_size].tolist() for i in range(0, len(chunk), titles_size)]

    with tqdm(total=len(title_groups), desc="Processing title groups") as pbar:
        with concurrent.futures.ThreadPoolExecutor() as executor:
            future_to_group = {executor.submit(process_titles, group): i for i, group in enumerate(title_groups)}

            for future in concurrent.futures.as_completed(future_to_group):
                group_index = future_to_group[future]
                try:
                    categories = future.result()
                    start_idx = group_index * titles_size
                    end_idx = min((group_index + 1) * titles_size, len(chunk))
                    if len(categories) == end_idx - start_idx:
                        for j, category in enumerate(categories):
                            results.append((chunk.index[start_idx + j], category))
                    else:
                        print(f"Warning: 그룹 {group_index+1}에서 카테고리 수가 제목 수와 일치하지 않습니다.")
                except Exception as exc:
                    print(f'그룹 {group_index}에서 오류 발생: {exc}')
                finally:
                    pbar.update(1)

    return results


청크 1 처리 시작
처리할 청크 크기: 10000 항목


Processing title groups:   1%|          | 7/1000 [00:01<03:40,  4.50it/s]

Processing title groups:  10%|▉         | 99/1000 [00:20<02:12,  6.80it/s]

Processing title groups:  17%|█▋        | 167/1000 [00:34<02:45,  5.03it/s]

Processing title groups:  22%|██▏       | 217/1000 [00:46<02:58,  4.38it/s]

...

Processing title groups:  47%|████▋     | 471/1000 [01:45<01:53,  4.66it/s]

Processing title groups:  63%|██████▎   | 631/1000 [02:21<01:27,  4.23it/s]

Processing title groups:  73%|███████▎  | 726/1000 [02:42<00:37,  7.32it/s]

Processing title groups: 100%|██████████| 1000/1000 [03:50<00:00,  4.33it/s]



청크 1 처리 완료. 청크의 첫 5개 행:
   ID                                              제목   카테고리
0   0         죽음은 죄의 결과인가?: 죄와 죽음의 관계 이해를 위한 교의신학적 성찰  기독교신학
1   1                  󰡔하느님을 찾는 사람들󰡕 예비 신자 교리서에 대한 소고  기독교신학
2   2              한국 천주교 교육사에서 바라보는 한국 살레시오회의 교육적 역할  기독교신학
3   3              연행 밀사 소통방식과 변화 - 천주교 밀사 기록을 중심으로 -  기독교신학
4   4  메리놀회의 중국 진입과 적응에 대한 초보적 탐색 -‘적응주의’와 ‘토착화’ 사이에서  기독교신학

처리된 청크를 classified_titles_chunk_1.csv로 저장 완료.

모든 청크 처리 완료.


In [ ]:
def main():
    # CSV 파일 읽기
    df = pd.read_csv('kci_titles.csv')

    # 청크 크기 설정
    titles_size = 10

    for chunk_num in range(1, len(my_chunks)+1):
        if chunk_number == len(my_chunks):
            chunk = df[130000:len(df)].copy()
        elif 1 <= chunk_number < 14:
            chunk_size = chunk_num * 10000

            # 지정된 청크 선택
            chunk = df[chunk_size-10000:chunk_size].copy()

        print(f"\n청크 {chunk_num} 처리 시작")
        print(f"처리할 청크 크기: {len(chunk)} 항목")

        # 단일 청크 처리
        results = process_chunk(chunk, titles_size,)

        # 결과를 청크에 적용
        chunk['카테고리'] = ''
        for idx, category in results:
            chunk.loc[idx, '카테고리'] = category

        # 결과 확인
        print(f"\n청크 {chunk_num} 처리 완료. 청크의 첫 5개 행:")
        print(chunk.head())

        # 처리된 청크만 CSV 파일로 저장
        output_filename = f'classified_titles_chunk_{chunk_num}.csv'
        chunk.to_csv(output_filename, index=False)
        print(f"\n처리된 청크를 {output_filename}로 저장 완료.")

    print("\n모든 청크 처리 완료.")

if __name__ == "__main__":
    main()

In [ ]:
# classified_titles_chunk_(1 to n).csv 를 classified_titles.csv로 통합
import glob

def merge_csv_files():
    # 통합할 CSV 파일들의 패턴
    pattern = 'classified_titles_chunk_*.csv'

    # 모든 청크 파일 리스트 가져오기
    all_files = glob.glob(pattern)

    # 파일들을 정렬하여 순서대로 처리
    all_files.sort(key=lambda x: int(x.split('_')[-1].split('.')[0]))

    # 결과를 저장할 빈 리스트
    df_list = []

    # 각 파일을 읽어서 리스트에 추가
    for filename in all_files:
        df = pd.read_csv(filename)
        df_list.append(df)
        print(f"파일 {filename}을 읽었습니다. 크기: {len(df)} 행")

    # 모든 데이터프레임을 하나로 결합
    combined_df = pd.concat(df_list, ignore_index=True)

    # 결합된 데이터를 새 CSV 파일로 저장
    output_filename = 'classified_titles.csv'
    combined_df.to_csv(output_filename, index=False)

    print(f"\n모든 청크 파일이 {output_filename}로 통합되었습니다.")
    print(f"통합된 파일의 총 행 수: {len(combined_df)}")

if __name__ == "__main__":
    merge_csv_files()

파일 classified_titles_chunk_1.csv을 읽었습니다. 크기: 10000 행
파일 classified_titles_chunk_2.csv을 읽었습니다. 크기: 10000 행
파일 classified_titles_chunk_3.csv을 읽었습니다. 크기: 10000 행
파일 classified_titles_chunk_4.csv을 읽었습니다. 크기: 10000 행
파일 classified_titles_chunk_5.csv을 읽었습니다. 크기: 10000 행
파일 classified_titles_chunk_6.csv을 읽었습니다. 크기: 10000 행
파일 classified_titles_chunk_7.csv을 읽었습니다. 크기: 10000 행
파일 classified_titles_chunk_8.csv을 읽었습니다. 크기: 10000 행
파일 classified_titles_chunk_9.csv을 읽었습니다. 크기: 10000 행
파일 classified_titles_chunk_10.csv을 읽었습니다. 크기: 10000 행
파일 classified_titles_chunk_11.csv을 읽었습니다. 크기: 10000 행
파일 classified_titles_chunk_12.csv을 읽었습니다. 크기: 10000 행
파일 classified_titles_chunk_13.csv을 읽었습니다. 크기: 10000 행
파일 classified_titles_chunk_14.csv을 읽었습니다. 크기: 3118 행

모든 청크 파일이 classified_titles.csv로 통합되었습니다.
통합된 파일의 총 행 수: 133118


# 2차 처리 (1개씩, 단일처리)

1차에서 분류 안 된 것들 분류하기

In [ ]:
# classified_titles.csv의 '카테고리' 열의 빈칸만 모아두는 df 만들기
# CSV 파일 읽기
df = pd.read_csv('classified_titles.csv')

# '카테고리' 열이 빈 값인 행만 선택
df_binkan = df[df['카테고리'].isna()]

# 결과 확인
print(df_binkan)

# 100개 미만의 데이터를 처리할 때

# Apply the classification function to each title
# 나중에 병렬처리로 바꿔보기
results = []
for index, row in tqdm(df_binkan.iterrows(), total=len(df_binkan)):
    title = row['제목']
    category = classify_kci_category(title)
    #df_binkan.at[index, '카테고리'] = category
    results.append((index, category))

            ID                                                 제목 카테고리
20320    20320  ‘민주화’의 역설과 노동시의 새로운 양상 - 민주화 이후 시대의 노동시와 한국 민주...  NaN
20321    20321                        <박제상 설화>의 전승적 측면에 따른 성격과 의의  NaN
20322    20322                     海港都市 木浦의 藝鄕性과 가요 <목포의 눈물> 활용사례  NaN
20323    20323  인민의 원한과 정치적인 것, 그리고 민주주의 - 식민지 정치소설들과의 연대의식 하에...  NaN
20324    20324            <대청전서> 런던대 SOAS도서관 소장본에 보이는 붉은 색 가필의 가치  NaN
...        ...                                                ...  ...
133113  133113                 전남 동부 지역의 마을 이름 연구 ― 옛 여천군을 중심으로 ―  NaN
133114  133114                                     옛지명 해석에 관한 문제들  NaN
133115  133115                                       일본 지명 표기의 특징  NaN
133116  133116                                땅이름(지명)의 자료와 우리말 연구  NaN
133117  133117                                       지명 차자 표기 해독법  NaN

[98 rows x 3 columns]


100%|██████████| 98/98 [01:00<00:00,  1.63it/s]


In [ ]:
# 결과를 데이터프레임에 적용
for idx, category in results:
    #print(idx)
    df.loc[idx, '카테고리'] = category

# 업데이트된 DataFrame을 새 CSV 파일로 저장
df.to_csv('classified_titles.csv', index=False)
print("\nCSV 파일로 저장 완료.")

20320
20321
20322
20323
20324
20325
20326
20327
20328
20329
29100
29101
29102
29103
29104
29105
29106
29107
29108
29109
37920
37921
37922
37923
37924
37925
37926
37927
37928
37929
39130
39131
39132
39133
39134
39135
39136
39137
39138
39139
44280
44281
44282
44283
44284
44285
44286
44287
44288
44289
72680
72681
72682
72683
72684
72685
72686
72687
72688
72689
96270
96271
96272
96273
96274
96275
96276
96277
96278
96279
115260
115261
115262
115263
115264
115265
115266
115267
115268
115269
130680
130681
130682
130683
130684
130685
130686
130687
130688
130689
133110
133111
133112
133113
133114
133115
133116
133117

CSV 파일로 저장 완료.


In [ ]:
# classified_titles_silhum_2.csv 에 빈칸이 있는지 확인
silhum = pd.read_csv('classified_titles.csv')
silhum[silhum['카테고리'].isna()] # 없다.

,ID,제목,카테고리


# xlsx 변환 및 드라이브 연동

In [ ]:
# 결과를 xlsx에 저장하려면 전각 문자를 없애야 한다.

import re

# CSV 파일 읽기
make_excel = pd.read_csv('classified_titles_silhum_2.csv')

# 특수 문자 및 특정 문자열 처리 함수
def replace_chars(text):
    # 입력값을 문자열로 변환
    text = str(text)

    # 전각 대시를 일반 대시로 대체
    text = text.replace('', '<')
    text = text.replace('', '>')
    text = text.replace('－', '-')

    return text

# '제목' 열에만 문자열 치환 적용
if '제목' in make_excel.columns:
    make_excel['제목'] = make_excel['제목'].astype(str).apply(replace_chars)
else:
    print("'제목' 열이 데이터프레임에 존재하지 않습니다.")

# XLSX 파일로 저장
make_excel.to_excel('classified_titles_silhum_2.xlsx', index=False, engine='openpyxl')
make_excel.to_csv('classified_titles_silhum_2.csv', index=False,)

print("변환이 완료되었습니다.")

변환이 완료되었습니다.


In [ ]:
# 생성한 .csv , .xlsx, .json 파일을 모두 드라이브에 다운로드

from google.colab import drive
from google.colab import files
import os

# 구글 드라이브 마운트
drive.mount('/content/drive')

# 파일을 저장할 구글 드라이브 경로 설정
save_path = "/content/drive/My Drive/Colab Notebooks/인문LLM/모델용"  # 원하는 경로로 수정 가능

# 현재 디렉토리의 모든 파일을 확인
for file in os.listdir():
    # .csv 또는 .xlsx 파일인 경우
    if file.endswith(('.csv', '.xlsx', '.json')):
        # 파일을 구글 드라이브로 복사
        !cp "{file}" "{save_path}"
        print(f"{file} 파일이 구글 드라이브의 {save_path} 경로에 저장되었습니다.")

print("모든 .csv 및 .xlsx 파일이 구글 드라이브에 저장되었습니다.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
classified_titles_chunk_3.csv 파일이 구글 드라이브의 /content/drive/My Drive/Colab Notebooks/인문LLM/모델용 경로에 저장되었습니다.
classified_titles_chunk_8.csv 파일이 구글 드라이브의 /content/drive/My Drive/Colab Notebooks/인문LLM/모델용 경로에 저장되었습니다.
df_binkan.csv 파일이 구글 드라이브의 /content/drive/My Drive/Colab Notebooks/인문LLM/모델용 경로에 저장되었습니다.
df_binkan_2.csv 파일이 구글 드라이브의 /content/drive/My Drive/Colab Notebooks/인문LLM/모델용 경로에 저장되었습니다.
classified_titles_chunk_1.csv 파일이 구글 드라이브의 /content/drive/My Drive/Colab Notebooks/인문LLM/모델용 경로에 저장되었습니다.
classified_titles_chunk_11.csv 파일이 구글 드라이브의 /content/drive/My Drive/Colab Notebooks/인문LLM/모델용 경로에 저장되었습니다.
classified_titles_chunk_13.csv 파일이 구글 드라이브의 /content/drive/My Drive/Colab Notebooks/인문LLM/모델용 경로에 저장되었습니다.
classified_titles_chunk_5.csv 파일이 구글 드라이브의 /content/drive/My Drive/Colab Notebooks/인문LLM/모델용 경로에 저장되었습니다.
classified_titles_silhum_2.xlsx 파일이 구글 드라이브의 /conte

# 카테고리 필터링


In [ ]:
# CSV 파일 읽기
df = pd.read_csv('classified_titles.csv')

# '카테고리'열에서 사회과학, 기독교신학, 문학, 언어학, 역사학, 철학, 통역번역학, 기타인문학, 관련없음만 추출
filtered_df = df[df['카테고리'].isin(['사회과학', '기독교신학', '문학', '언어학', '역사학', '철학', '통역번역학', '기타인문학', '관련없음'])]

# filtered_df에서 '사회과학', '기독교신학'을 뺀 filtered_more_df
filtered_more_df = filtered_df[~filtered_df['카테고리'].isin(['사회과학', '기독교신학'])]

# df와 filtered_df의 개수와 비교
print(f"df의 행 개수: {len(df)}")
print(f"filtered_df의 행 개수: {len(filtered_df)}")
print(f"filtered_more_df의 행 개수: {len(filtered_more_df)}")

df의 행 개수: 133118
filtered_df의 행 개수: 131464
filtered_more_df의 행 개수: 108626


In [ ]:
# df에 존재하는 모든 카테고리들의 len을 print해줘. lambda 써서 빈도 순으로
bindo = {category: len(df[df['카테고리'] == category]) for category in df['카테고리'].unique()}
bindo = sorted(bindo.items(), key=lambda x: x[1], reverse=True)
bindo

# # 416 이하의 빈도는 모두 더해서 print 해줘
# total = sum(count for category, count in bindo if count <= 416)
# #print(total)

[('문학', 29559),
 ('역사학', 28970),
 ('기타인문학', 20109),
 ('철학', 15176),
 ('사회과학', 12418),
 ('언어학', 11555),
 ('기독교신학', 10420),
 ('관련없음', 2165),
 ('통역번역학', 1092),
 ('교육학', 416),
 ('영화학', 121),
 ('문화연구', 120),
 ('심리학', 86),
 ('법학', 82),
 ('신학', 56),
 ('민속학', 56),
 ('미술사', 53),
 ('미술', 46),
 ('고고학', 41),
 ('문화인류학', 38),
 ('사회학', 35),
 ('정치학', 31),
 ('여성학', 25),
 ('종교학', 24),
 ('예술학', 24),
 ('영화연구', 18),
 ('예술', 18),
 ('경제학', 17),
 ('번역학', 15),
 ('음악학', 14),
 ('미학', 14),
 ('미디어학', 14),
 ('군사학', 11),
 ('음악', 10),
 ('연극학', 9),
 ('문학이론', 9),
 ('문화재학', 8),
 ('문헌학', 7),
 ('지리학', 6),
 ('문화학', 6),
 ('문화콘텐츠', 6),
 ('의료윤리', 6),
 ('사회복지학', 5),
 ('인문학', 5),
 ('환경학', 5),
 ('신화학', 5),
 ('건축학', 5),
 ('불교학', 5),
 ('수학', 5),
 ('윤리학', 4),
 ('성경학', 4),
 ('문화사', 4),
 ('관계없음', 4),
 ('의학', 4),
 ('기독교교육학', 3),
 ('상담학', 3),
 ('정보학', 3),
 ('수사학', 3),
 ('중국학', 3),
 ('관광학', 3),
 ('국제관계', 3),
 ('박물관학', 3),
 ('물리학', 3),
 ('과학철학', 3),
 ('상담심리학', 2),
 ('심리학과 신학의 관계는 인문학적 접근으로 볼 수 있으나, 주제에 따라 기독교신학으로 분류할 수 있음', 2),
 ('디지털 인문

In [ ]:
filtered_more_df.to_csv('filtered_more_df.csv', index=False)
filtered_more_df.to_excel('filtered_more_df.xlsx', index=False)

In [ ]:
import json
import pandas as pd

# CSV 파일에서 id 목록 읽기
df = pd.read_csv('filtered_more_df.csv')
valid_ids = set(df['ID'].astype(str))  # id를 문자열로 변환

# JSON 파일 읽기
with open('crawl_kci.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# 유효한 id만 남기기
filtered_data = {k: v for k, v in data.items() if k in valid_ids}
print(len(filtered_data))

# 결과 저장
with open('filtered_crawl_kci.json', 'w', encoding='utf-8') as f:
    json.dump(filtered_data, f, ensure_ascii=False, indent=2)

108626
